In [1]:
# baseline 0.95214

In [2]:
# Logit Tf-Idf 6 features

In [3]:
# from Correct time-aware cross-validation scheme

In [4]:
# Import libraries and set desired options
import numpy as np
import pandas as pd
#from scipy.sparse import hstack
from scipy.sparse import csr_matrix, hstack
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

In [5]:
from sklearn.preprocessing import StandardScaler

In [6]:
# A helper function for writing predictions to a file
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [7]:
train_df = pd.read_csv('train_sessions.csv',
                       index_col='session_id')
test_df = pd.read_csv('test_sessions.csv',
                      index_col='session_id')

# Convert time1, ..., time10 columns to datetime type
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# Sort the data by time
train_df = train_df.sort_values(by='time1')

# Look at the first rows of the training set
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


In [8]:
sites = ['site%s' % i for i in range(1, 11)]
train_df[sites].fillna(0).astype('int').to_csv('train_sessions_text.txt', 
                                               sep=' ', index=None, header=None)
test_df[sites].fillna(0).astype('int').to_csv('test_sessions_text.txt', 
                                              sep=' ', index=None, header=None)

In [9]:
!head -5 train_sessions_text.txt

56 55 0 0 0 0 0 0 0 0
56 55 56 55 0 0 0 0 0 0
946 946 951 946 946 945 948 784 949 946
945 948 949 948 945 946 947 945 946 946
947 950 948 947 950 952 946 951 946 947


In [10]:
%%time
cv = CountVectorizer(ngram_range=(1, 3), max_features=50000)
with open('train_sessions_text.txt') as inp_train_file:
    X_train = cv.fit_transform(inp_train_file)
with open('test_sessions_text.txt') as inp_test_file:
    X_test = cv.transform(inp_test_file)
X_train.shape, X_test.shape

Wall time: 13.7 s


In [11]:
y_train = train_df['target'].astype('int')

In [12]:
time_split = TimeSeriesSplit(n_splits=10)

In [13]:
[(el[0].shape, el[1].shape) for el in time_split.split(X_train)]

[((23051,), (23051,)),
 ((46102,), (23051,)),
 ((69153,), (23051,)),
 ((92204,), (23051,)),
 ((115255,), (23051,)),
 ((138306,), (23051,)),
 ((161357,), (23051,)),
 ((184408,), (23051,)),
 ((207459,), (23051,)),
 ((230510,), (23051,))]

In [14]:
logit = LogisticRegression(C=1, random_state=17)

In [15]:
%%time

cv_scores = cross_val_score(logit, X_train, y_train, cv=time_split, 
                            scoring='roc_auc', n_jobs=-1) # hangs with n_jobs > 1, and locally this runs much faster

Wall time: 44.9 s


In [16]:
cv_scores, cv_scores.mean()

(array([0.83141992, 0.64669477, 0.87991917, 0.96315292, 0.84221701,
        0.87840872, 0.94475732, 0.85321644, 0.92987908, 0.90752818]),
 0.8677193536148717)

In [17]:
logit.fit(X_train, y_train)

d:\program files\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=17, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [18]:
logit_test_pred = logit.predict_proba(X_test)[:, 1]
write_to_submission_file(logit_test_pred, 'subm1.csv') # 0.91288

In [19]:
def add_time_features(df, X_sparse):
    hour = df['time1'].apply(lambda ts: ts.hour)
    morning = ((hour >= 7) & (hour <= 11)).astype('int')
    day = ((hour >= 12) & (hour <= 18)).astype('int')
    evening = ((hour >= 19) & (hour <= 23)).astype('int')
    night = ((hour >= 0) & (hour <= 6)).astype('int')
    X = hstack([X_sparse, morning.values.reshape(-1, 1), 
                day.values.reshape(-1, 1), evening.values.reshape(-1, 1), 
                night.values.reshape(-1, 1)])
    return X

In [20]:
%%time
X_train_new = add_time_features(train_df.fillna(0), X_train)
X_test_new = add_time_features(test_df.fillna(0), X_test)

Wall time: 1min 44s


In [21]:
X_train_new.shape, X_test_new.shape

((253561, 50004), (82797, 50004))

In [22]:
%%time
cv_scores = cross_val_score(logit, X_train_new, y_train, cv=time_split, 
                            scoring='roc_auc', n_jobs=-1) # hangs with n_jobs > 1, and locally this runs much faster

Wall time: 43.6 s


In [23]:
cv_scores, cv_scores.mean()

(array([0.87652278, 0.75122979, 0.93062182, 0.97864617, 0.90399606,
        0.93831379, 0.96248761, 0.92731291, 0.94885607, 0.94043603]),
 0.915842304479402)

In [24]:
logit.fit(X_train_new, y_train)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=17, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [25]:
logit_test_pred2 = logit.predict_proba(X_test_new)[:, 1]
write_to_submission_file(logit_test_pred2, 'subm2.csv') # 0.93843

In [26]:
c_values = np.logspace(-2, 2, 10)

logit_grid_searcher = GridSearchCV(estimator=logit, param_grid={'C': c_values},
                                  scoring='roc_auc', n_jobs=-1, cv=time_split, verbose=1)

In [27]:
%%time
logit_grid_searcher.fit(X_train_new, y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   38.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  8.7min finished


Wall time: 9min


GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=10),
       error_score='raise-deprecating',
       estimator=LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=17, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'C': array([1.00000e-02, 2.78256e-02, 7.74264e-02, 2.15443e-01, 5.99484e-01,
       1.66810e+00, 4.64159e+00, 1.29155e+01, 3.59381e+01, 1.00000e+02])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=1)

In [28]:
logit_grid_searcher.best_score_, logit_grid_searcher.best_params_

(0.9173776926166856, {'C': 0.21544346900318834})

In [29]:
#logit_test_pred3 = logit_grid_searcher.predict_proba(X_test_new)[:, 1]
#write_to_submission_file(logit_test_pred3, 'subm3.csv') # 0.94242

In [30]:
#GridSearchCV?

In [31]:
def add_day_features(df, X_sparse):
    day = df['time1'].apply(lambda ts: ts.weekday())
    weekend = ((day >= 5)).astype('int')
    weekday = ((day < 5)).astype('int')
    X = hstack([X_sparse, weekend.values.reshape(-1, 1), 
                weekday.values.reshape(-1, 1)])
    return X

In [32]:
%%time
X_train_new1 = add_time_features(train_df.fillna(0), X_train_new)
X_test_new1 = add_time_features(test_df.fillna(0), X_test_new)

Wall time: 1min 43s


In [33]:
#X_train_new1.shape, X_test_new1.shape

In [35]:
#%%time
#cv_scores = cross_val_score(logit, X_train_new1, y_train, cv=time_split, 
#                            scoring='roc_auc', n_jobs=-1) # hangs with n_jobs > 1, and locally this runs much faster

In [36]:
#cv_scores, cv_scores.mean()
# 0.9164082065409461

In [37]:
#%%time
#logit_grid_searcher.fit(X_train_new1, y_train)

In [38]:
#logit_grid_searcher.best_score_, logit_grid_searcher.best_params_
# (0.9183503264075926, {'C': 0.21544346900318834})

In [39]:
#logit_test_pred4 = logit_grid_searcher.predict_proba(X_test_new1)[:, 1]
#write_to_submission_file(logit_test_pred4, 'subm_self1.csv')

In [40]:
def label_unique_site_sum(row):
    s = set()
    for i in sites:
        if row[i] != 0:
            s.add(row[i])
            pass
        pass
    return sum(s)

In [41]:
def label_unique_site(row):
    s = set()
    for i in sites:
        if row[i] != 0:
            s.add(row[i])
            pass
        pass
    return len(s)

In [42]:
def label_unique_site_sum(row):
    s = set()
    for i in sites:
        if row[i] != 0:
            s.add(row[i])
            pass
        pass
    return sum(s)

In [43]:
def label_unique_site(row):
    s = set()
    for i in sites:
        if row[i] != 0:
            s.add(row[i])
            pass
        pass
    return len(s)

In [44]:
### 7

In [45]:
#### 8

In [46]:
### no more use the original data nummber

In [47]:
#train_df.head()

In [48]:
alice_web_statistic = {}
alice_web_count_statistic = {}

def getalice_feature(row):
    s = set()
    for i in sites:
        webid = row[i]
        s.add(webid)
        if webid in alice_web_statistic:
            alice_web_statistic[webid] = alice_web_statistic[webid] + 1
        else:
            alice_web_statistic[webid] = 1
            pass
        pass
    webcount = len(s)
    if webcount in alice_web_count_statistic:
        alice_web_count_statistic[webcount] = alice_web_count_statistic[webcount] + 1
    else:
        alice_web_count_statistic[webcount] = 1
        pass
    pass

#train_df[train_df['target'] == 1][sites].fillna(0).astype('int').apply(lambda row: getalice_feature(row),axis=1)

In [49]:
#alice_web_count_statistic

In [50]:
#alice_web_statistic

In [51]:
alice_web_count = [(k, alice_web_count_statistic[k]) for k in sorted(alice_web_count_statistic, key=alice_web_count_statistic.get, reverse=True)][: 5]

In [52]:
alice_love_web = [(k, alice_web_statistic[k]) for k in sorted(alice_web_statistic, key=alice_web_statistic.get, reverse=True)][: 10]

In [53]:
#alice_web_statistic[0]

In [54]:
def match_alice_love_web(row):
    count = 0
    for i in sites:
        if row[i] in alice_love_web:
            count += 1
            pass
        pass
    return count > 5

In [55]:
def match_alice_web_count(row):
    s = set()
    for i in sites:
        if row[i] != 0:
            s.add(row[i])
            pass
        pass
    return sum(s) in alice_web_count

In [56]:
def add_features8(df, X_sparse):
    hour = df['time1'].apply(lambda ts: ts.hour)
    morning = ((hour >= 7) & (hour <= 11)).astype('int')
    day = ((hour >= 12) & (hour <= 18)).astype('int')
    evening = ((hour >= 19) & (hour <= 23)).astype('int')
    night = ((hour >= 0) & (hour <= 6)).astype('int')
    
    dayinweek = df['time1'].apply(lambda ts: ts.weekday())
    weekend = ((dayinweek >= 5)).astype('int')
    weekday = ((dayinweek < 5)).astype('int')
    
    web_count_match = df.apply (lambda row: match_alice_web_count (row),axis=1).astype('int')
    
    web_type = df.apply (lambda row: match_alice_love_web (row),axis=1)
   
    
    X = hstack([X_sparse, morning.values.reshape(-1, 1), 
                day.values.reshape(-1, 1), evening.values.reshape(-1, 1), 
                night.values.reshape(-1, 1)
                , weekend.values.reshape(-1, 1)
                , weekday.values.reshape(-1, 1)
                , web_count_match.values.reshape(-1, 1)
                , web_type.values.reshape(-1, 1)
               ])
    return X

In [57]:
#%%time
#X_train_new8 = add_features8(train_df.fillna(0), X_train)
#X_test_new8 = add_features8(test_df.fillna(0), X_test)

In [58]:
#%%time
#cv_scores = cross_val_score(fastlogist, X_train_new8, y_train, cv=time_split, 
#                            scoring='roc_auc', n_jobs=2) # hangs with n_jobs > 1, and locally this runs much faster

In [59]:
#cv_scores, cv_scores.mean()
# 0.914169880747776

In [60]:
#%%time
#fastlogist.fit(X_train_new8, y_train)

In [61]:
#logit_test_pred8 = fastlogist.predict_proba(X_test_new8)[:, 1]
#write_to_submission_file(logit_test_pred8, 'subm_self8.csv')

In [62]:
#%%time
#logit_grid_searcher.fit(X_train_new8, y_train)

In [63]:
#logit_grid_searcher.best_score_, logit_grid_searcher.best_params_
# (0.9081530648461433, {'C': 0.21544346900318834})

In [64]:
### 9

In [65]:

alice_web_count_score = {}

count = 0
for key, val in alice_web_count_statistic.items():
    count += val
    pass
for key, val in alice_web_count_statistic.items():
    alice_web_count_score[key] = val * 1.0 / count
    pass

In [66]:

alice_web_score = {}
count = 0

for key, val in alice_web_statistic.items():
    count += val
    pass
for key, val in alice_web_statistic.items():
    alice_web_score[key] = val * 1.0 / count
    pass

In [67]:
def get_web_score(row):
    score = 0
    for i in sites:
        if row[i] != 0:
            if row[i] in alice_web_count_score:
                score += alice_web_count_score[row[i]]
            pass
        pass
    return score
    pass

In [68]:
def get_web_count_score(row):
    count = 0
    for i in sites:
        if row[i] != 0:
            count += 1
            pass
        pass
    if count in alice_web_count_score:
        return alice_web_count_score[count]
    else:
        return 0
    pass

In [69]:
## 10

In [70]:
#np.linspace(0.1, 1, 10)

In [71]:
#logit_grid_searcher3 = GridSearchCV(estimator=logit, param_grid={'C': np.linspace(0.1, 1, 10)},
#                                  scoring='accuracy', n_jobs=-1, cv=time_split, verbose=1)

In [72]:
#logit_grid_searcher3.fit(X_train_new10, y_train)

In [73]:
#logit_grid_searcher3.best_score_, logit_grid_searcher3.best_params_

In [74]:
#logit_test_pred10 = logit_grid_searcher3.predict_proba(X_test_new10)[:, 1]
#write_to_submission_file(logit_test_pred10, 'subm_self10.csv')

In [75]:
#logit_grid_searcher4 = GridSearchCV(estimator=logit, param_grid={'C': c_values},
#                                  scoring='accuracy', n_jobs=-1, cv=time_split, verbose=1)

In [76]:
#logit_grid_searcher4.fit(X_train_new10, y_train)

In [77]:
#logit_grid_searcher4.best_score_, logit_grid_searcher4.best_params_

In [78]:
#logit_test_pred11 = logit_grid_searcher4.predict_proba(X_test_new10)[:, 1]
#write_to_submission_file(logit_test_pred11, 'subm_self11.csv')

In [79]:
#np.logspace(-2, 2, 10)

In [80]:
#np.logspace(-3, 1, 10)

In [81]:
#c_values = np.logspace(-3, 1, 10)

#logit_grid_searcher_1 = GridSearchCV(estimator=logit, param_grid={'C': c_values},
#                                  scoring='roc_auc', n_jobs=-1, cv=time_split, verbose=1)

In [82]:
#logit_grid_searcher_1.fit(X_train_new10, y_train)

In [83]:
#logit_grid_searcher_1.best_score_, logit_grid_searcher_1.best_params_
# 0.913374049587434

In [84]:
#%%time
#logit_grid_searcher_1.fit(X_train_new1, y_train)

In [85]:
#logit_grid_searcher_1.best_score_, logit_grid_searcher_1.best_params_
# 0.9181526261645014

In [86]:
#np.linspace(0.15, 0.25, 10)

In [87]:
#logit_grid_searcher_2 = GridSearchCV(estimator=logit, param_grid={'C': np.linspace(0.15, 0.25, 10)},
#                                  scoring='roc_auc', n_jobs=-1, cv=time_split, verbose=1)

In [88]:
#%%time
#logit_grid_searcher_2.fit(X_train_new1, y_train)

In [89]:
#logit_grid_searcher_2.best_score_, logit_grid_searcher_2.best_params_
# (0.9183729937045094, {'C': 0.22777777777777777})

In [90]:
#logit_grid_searcher_3 = GridSearchCV(estimator=logit, param_grid={'C': np.linspace(0.22, 0.6, 20)},
#                                  scoring='roc_auc', n_jobs=-1, cv=time_split, verbose=1)

In [91]:
#%%time
#logit_grid_searcher_3.fit(X_train_new1, y_train)

In [92]:
#logit_grid_searcher_3.best_score_, logit_grid_searcher_3.best_params_
# (0.9183505761017501, {'C': 0.24})

In [93]:
#logit_grid_searcher_4 = GridSearchCV(estimator=logit, param_grid={'C': np.linspace(0.22, 0.25, 10)},
#                                  scoring='roc_auc', n_jobs=-1, cv=time_split, verbose=1)

In [94]:
#%%time
#logit_grid_searcher_4.fit(X_train_new1, y_train)

In [95]:
#logit_grid_searcher_4.best_score_, logit_grid_searcher_4.best_params_

In [ ]:
#logit_grid_searcher_5 = GridSearchCV(estimator=logit, param_grid={'C': np.linspace(0.223, 0.229, 10)},
#                                  scoring='roc_auc', n_jobs=-1, cv=time_split, verbose=1)

In [ ]:
#%%time
#logit_grid_searcher_5.fit(X_train_new1, y_train)

In [ ]:
#logit_grid_searcher_5.best_score_, logit_grid_searcher_5.best_params_

In [ ]:
#logit_test_pred12 = logit_grid_searcher_5.predict_proba(X_test_new1)[:, 1]
#write_to_submission_file(logit_test_pred12, 'subm_self12.csv')

In [ ]:
### 11

In [ ]:
### feature day_of_week, web_unique, time_diff1-10, total_time, 

In [ ]:
### inspire by https://www.kaggle.com/adityaecdrid/initial-eda and https://www.kaggle.com/hakeydotcom/additional-time-features-and-logit

In [ ]:
### optional stop hour

In [96]:
def get_web_unique(row):
    s = set()
    for i in sites:
        if row[i] != 0:
            s.add(row[i])
            pass
        pass
    return len(s)

In [97]:
def get_total_time(row):
    if len(times) <= 1:
        return 0
    start = row[times[0]]
    end = start
    for i in times:
        if row[i] == 0:
            break
            pass
        end = row[i]
        pass
    return (end - start) / np.timedelta64(1, 's')

In [98]:
def get_diff_time(row, f, t):
    if len(times) < f or len(times) < t:
        return 0
    if row[times[f]] == 0 or row[times[t]] == 0:
        return 0
    return (row[times[t]] - row[times[f]]) / np.timedelta64(1, 's')

In [99]:
def add_features_11(df, X_sparse):
    dayinweek = df['time1'].apply(pd.datetime.weekday)
    web_unique = df.apply(lambda row: get_web_unique(row), axis=1)
    total_time = df.apply(lambda row: get_total_time(row), axis=1)
    diff_time1 = df.apply(lambda row: get_diff_time(row, 1, 2), axis=1)
    diff_time2 = df.apply(lambda row: get_diff_time(row, 2, 3), axis=1)
    diff_time3 = df.apply(lambda row: get_diff_time(row, 3, 4), axis=1)
    diff_time4 = df.apply(lambda row: get_diff_time(row, 4, 5), axis=1)
    diff_time5 = df.apply(lambda row: get_diff_time(row, 5, 6), axis=1)
    diff_time6 = df.apply(lambda row: get_diff_time(row, 6, 7), axis=1)
    diff_time7 = df.apply(lambda row: get_diff_time(row, 7, 8), axis=1)
    diff_time8 = df.apply(lambda row: get_diff_time(row, 8, 9), axis=1)
    diff_time9 = df.apply(lambda row: get_diff_time(row, 9, 10), axis=1)
    
    scaler = StandardScaler() 
    web_unique = scaler.fit_transform(web_unique.values.reshape(-1, 1))
    total_time = scaler.fit_transform(total_time.values.reshape(-1, 1))
    diff_time1 = scaler.fit_transform(diff_time1.values.reshape(-1, 1))
    diff_time2 = scaler.fit_transform(diff_time2.values.reshape(-1, 1))
    diff_time3 = scaler.fit_transform(diff_time3.values.reshape(-1, 1))
    diff_time4 = scaler.fit_transform(diff_time4.values.reshape(-1, 1))
    diff_time5 = scaler.fit_transform(diff_time5.values.reshape(-1, 1))
    diff_time6 = scaler.fit_transform(diff_time6.values.reshape(-1, 1))
    diff_time7 = scaler.fit_transform(diff_time7.values.reshape(-1, 1))
    diff_time8 = scaler.fit_transform(diff_time8.values.reshape(-1, 1))
    diff_time9 = scaler.fit_transform(diff_time9.values.reshape(-1, 1))
    
    X = hstack([X_sparse
                , dayinweek.values.reshape(-1, 1)
                , web_unique[:, :]
                , total_time[:, :]
                , diff_time1[:, :]
                , diff_time2[:, :]
                , diff_time3[:, :]
                , diff_time4[:, :]
                , diff_time5[:, :]
                , diff_time6[:, :]
                , diff_time7[:, :]
                , diff_time8[:, :]
                , diff_time9[:, :]
               ])
    return X

In [100]:
#%%time
#X_train_new11 = add_time_features(train_df.fillna(0), X_train_new)
#X_test_new11 = add_time_features(test_df.fillna(0), X_test_new)

Wall time: 1min 45s


In [103]:
#%%time
#cv_scores = cross_val_score(logit, X_train_new11, y_train, cv=time_split, 
#                            scoring='roc_auc', n_jobs=-1) # hangs with n_jobs > 1, and locally this runs much faster

Wall time: 47.2 s


In [104]:
#cv_scores, cv_scores.mean()

(array([0.87806863, 0.75450051, 0.93075898, 0.97850289, 0.90417853,
        0.9384806 , 0.96237984, 0.92768015, 0.94890392, 0.94062803]),
 0.9164082065409461)

In [110]:
#X_test_new11

<82797x50008 sparse matrix of type '<class 'numpy.int64'>'
	with 1178796 stored elements in COOrdinate format>

In [111]:
#X_train_new11

<253561x50008 sparse matrix of type '<class 'numpy.int64'>'
	with 3886675 stored elements in COOrdinate format>

In [112]:
#logit.fit(X_train_new11, y_train)

d:\program files\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=17, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [115]:
#logit_test_pred14 = logit.predict_proba(X_test_new11)[:, 1]
#write_to_submission_file(logit_test_pred14, 'subm14.csv') # 0.93860

In [105]:
#c_values11 = np.logspace(-2, 2, 20)

#logit_grid_searcher11 = GridSearchCV(estimator=logit, param_grid={'C': c_values11},
#                                  scoring='roc_auc', n_jobs=-1, cv=time_split, verbose=1)

In [106]:
#%%time
#logit_grid_searcher11.fit(X_train_new11, y_train)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   34.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 14.3min
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed: 18.2min finished
d:\program files\python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Wall time: 18min 33s


GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=10),
       error_score='raise-deprecating',
       estimator=LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=17, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'C': array([1.00000e-02, 1.62378e-02, 2.63665e-02, 4.28133e-02, 6.95193e-02,
       1.12884e-01, 1.83298e-01, 2.97635e-01, 4.83293e-01, 7.84760e-01,
       1.27427e+00, 2.06914e+00, 3.35982e+00, 5.45559e+00, 8.85867e+00,
       1.43845e+01, 2.33572e+01, 3.79269e+01, 6.15848e+01, 1.00000e+02])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=1)

In [107]:
#logit_grid_searcher11.best_score_, logit_grid_searcher11.best_params_
#(0.9182907311098205, {'C': 0.29763514416313175})

(0.9182907311098205, {'C': 0.29763514416313175})

In [117]:
#logit_test_pred15 = logit_grid_searcher11.predict_proba(X_test_new11)[:, 1]
#write_to_submission_file(logit_test_pred15, 'subm_self15.csv') # 0.94217